In [ ]:
import json
import os
import pathlib
import random
import time
import uuid
import numpy as np

from bs4 import BeautifulSoup
from google.colab import drive
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import altair as alt
from altair import datum
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
DATA_PATH_FINAL = '/content/gdrive/MyDrive/MIDS/capstone'

# User data Load: This will be only available for the last 3 seasons


In [ ]:
user_input = pd.read_csv(f"{DATA_PATH_FINAL}/urser_input.csv",keep_default_na=False)

In [ ]:
user_input

,date_time,combined_terrain_aspects,combined_terrain_elevations
0,2021-04-17,S,Above Treeline
1,2021-04-17,S,Near Treeline
2,2021-04-17,S,Below Treeline
3,2021-04-17,NE,Above Treeline
4,2021-04-17,NE,Near Treeline
...,...,...,...
4665,2022-11-30,W,Above Treeline
4666,2022-11-30,W,Near Treeline
4667,2022-11-30,W,Below Treeline
4668,2022-11-30,N,Below Treeline


# Load additional features (TO DO: I will work on the current season features engineering )

In [ ]:
additional_feature = pd.read_csv(f"{DATA_PATH_FINAL}/additional_feature.csv",keep_default_na=False)

In [ ]:
additional_feature.shape

(421, 3156)

In [ ]:
additional_feature[additional_feature['date_time'] == '2021-04-17']

,date_time,above_treeline_cat,near_treeline_cat,below_treeline_cat,likelihood_0_cat,likelihood_1_cat,likelihood_2_cat,size_0_cat,size_1_cat,size_2_cat,...,problem_discussion_2_bert_all_case_emb_758,problem_discussion_2_bert_all_case_emb_759,problem_discussion_2_bert_all_case_emb_760,problem_discussion_2_bert_all_case_emb_761,problem_discussion_2_bert_all_case_emb_762,problem_discussion_2_bert_all_case_emb_763,problem_discussion_2_bert_all_case_emb_764,problem_discussion_2_bert_all_case_emb_765,problem_discussion_2_bert_all_case_emb_766,problem_discussion_2_bert_all_case_emb_767
0,2021-04-17,2.0,2.0,2.0,2.0,-1.0,-1.0,3.0,-1.0,-1.0,...,0.158837,0.583574,-1.081375,-0.569019,0.808689,-0.241232,-0.506336,-0.373096,-0.38232,0.336898


# Concat User information + additional features

In [ ]:
all_features = pd.merge(additional_feature, user_input, on='date_time', how='inner')

In [ ]:
all_features.shape

(4670, 3158)

# Convert the dataframe to datasets

In [ ]:
import tensorflow as tf

def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels = None  # Initialize labels as None

    # Remove the target column if it exists
    if 'target' in df.columns:
        labels = df.pop('target')  # Remove and store the target column

    # Convert the dataframe to a dictionary of lists
    df_dict = {key: value.tolist() for key, value in df.items()}

    # Create a dataset from the dictionary and labels
    ds = tf.data.Dataset.from_tensor_slices((df_dict, labels))

    # Shuffle the dataset if specified
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))

    # Batch and prefetch the dataset
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)

    return ds


In [ ]:
ds = df_to_dataset(all_features, False)

# Load Model and predict

In [ ]:
model = tf.keras.models.load_model(f"{DATA_PATH_FINAL}/tf_rand_val_nn_v1")

In [ ]:
model.predict(ds)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['date_time'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


146/146 [==============================] - 60s 145ms/step


array([[0.02322091],
       [0.02134966],
       [0.02007184],
       ...,
       [0.01201669],
       [0.01129102],
       [0.01129102]], dtype=float32)